## Import

In [1]:
import random
import pandas as pd
import numpy as np
import os
from tqdm.auto import tqdm
import librosa

from sklearn.tree import DecisionTreeClassifier

import warnings
warnings.filterwarnings(action='ignore') 

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/음성 분류

/content/drive/MyDrive/음성 분류


In [4]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 106.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 103.9 MB/s eta 0:00:00


In [5]:
import torch
import librosa
import numpy as np
import soundfile as sf
from scipy.io import wavfile
from IPython.display import Audio
from transformers import Wav2Vec2ForCTC, Wav2Vec2Tokenizer

## Hyperparameter Setting

In [6]:
CFG = {
    'SR':16000,
    'N_MFCC':32, # Melspectrogram 벡터를 추출할 개수
    'SEED':42
}

## Fixed Random-Seed

In [7]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(CFG['SEED']) # Seed 고정

## Data Pre-Processing

In [8]:
train_df = pd.read_csv('/content/drive/MyDrive/음성 분류/train.csv')
test_df = pd.read_csv('/content/drive/MyDrive/음성 분류/test.csv')

In [9]:
tokenizer = Wav2Vec2Tokenizer.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Wav2Vec2CTCTokenizer'. 
The class this function is called from is 'Wav2Vec2Tokenizer'.


Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
### train data 음성 인식 진행하기

for train_num in range(0, 5001):
    file_name = f"/content/drive/MyDrive/음성 분류/train/TRAIN_{str(train_num).zfill(4)}.wav"

    data = wavfile.read(file_name)
    framerate = data[0]
    sounddata = data[1]
    time = np.arange(0,len(sounddata))/framerate

    input_audio, _ = librosa.load(file_name, sr=16000)

    input_values = tokenizer(input_audio, return_tensors="pt").input_values
    logits = model(input_values).logits
    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = tokenizer.batch_decode(predicted_ids)[0]
    train_df.loc[train_num, "transcription"]=transcription

In [21]:
### test data 음성 인식 진행하기

for test_num in range(0, 1881):
    file_name = f"/content/drive/MyDrive/음성 분류/test/TEST_{str(test_num).zfill(4)}.wav"

    data = wavfile.read(file_name)
    framerate = data[0]
    sounddata = data[1]
    time = np.arange(0,len(sounddata))/framerate

    input_audio, _ = librosa.load(file_name, sr=16000)

    input_values = tokenizer(input_audio, return_tensors="pt").input_values
    logits = model(input_values).logits
    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = tokenizer.batch_decode(predicted_ids)[0]
    test_df.loc[test_num, "transcription"]=transcription

In [22]:
train_df

,id,path,label,transcription
0,TRAIN_0000,./train/TRAIN_0000.wav,1,IT'S A AT A COCK
1,TRAIN_0001,./train/TRAIN_0001.wav,2,THE SERVICE IS SLICK
2,TRAIN_0002,./train/TRAIN_0002.wav,4,WE'LL STOP IN A COUPLE OF MINUTES
3,TRAIN_0003,./train/TRAIN_0003.wav,5,MAYBE TO MORROW IT WILL BE COLD
4,TRAIN_0004,./train/TRAIN_0004.wav,4,DON'T FORGET A JACKET
...,...,...,...,...
4996,TRAIN_4996,./train/TRAIN_4996.wav,5,MAYBE TO MORROW IT WILL BE COLD
4997,TRAIN_4997,./train/TRAIN_4997.wav,0,IT'S ELEVEN O'CLOCK
4998,TRAIN_4998,./train/TRAIN_4998.wav,1,I WONDER WHAT THIS IS ABOUT
4999,TRAIN_4999,./train/TRAIN_4999.wav,1,THAT IS EXACTLY WHAT HAPPENED


In [23]:
test_df

,id,path,transcription
0,TEST_0000,./test/TEST_0000.wav,I WOULD LIKE A NEW ALARM CLOCK
1,TEST_0001,./test/TEST_0001.wav,MAYBE TO MORROW IT WILL BE COLD
2,TEST_0002,./test/TEST_0002.wav,MAYBE TOMORROW IT WILL BE COLD
3,TEST_0003,./test/TEST_0003.wav,IT'S ELEVEN O'CLOCK
4,TEST_0004,./test/TEST_0004.wav,DON'T FORGET A JACKET
...,...,...,...
1876,TEST_1876,./test/TEST_1876.wav,THE SURFACE IS SLICK
1877,TEST_1877,./test/TEST_1877.wav,IT'S ELEVEN O'CLOCK
1878,TEST_1878,./test/TEST_1878.wav,IT'S ELEVEN O'CLOCK
1879,TEST_1879,./test/TEST_1879.wav,I WONDER WHAT THIS IS ABOUT


In [24]:
# file_name = '/content/drive/MyDrive/음성 분류/train/TRAIN_0000.wav' 
# Audio(file_name)

In [25]:
# data = wavfile.read(file_name)
# framerate = data[0]
# sounddata = data[1]
# time = np.arange(0,len(sounddata))/framerate
# print(data)
# print('Sampling rate:',framerate,'Hz')

In [26]:
# input_audio, _ = librosa.load(file_name, sr=16000)

In [27]:
# input_values = tokenizer(input_audio, return_tensors="pt").input_values
# logits = model(input_values).logits
# predicted_ids = torch.argmax(logits, dim=-1)
# transcription = tokenizer.batch_decode(predicted_ids)[0]
# print(transcription)

In [28]:
# def get_mfcc_feature(df):
#     features = []
#     for path in tqdm(df['path']):
#         # librosa패키지를 사용하여 wav 파일 load
#         y, sr = librosa.load(path, sr=CFG['SR'])
#         # librosa패키지를 사용하여 mfcc 추출
#         mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=CFG['N_MFCC'])
#         y_feature = []
#         # 추출된 MFCC들의 평균을 Feature로 사용
#         for e in mfcc:
#             y_feature.append(np.mean(e))
#         features.append(y_feature)

#     mfcc_df = pd.DataFrame(features, columns=['mfcc_'+str(x) for x in range(1,CFG['N_MFCC']+1)])
#     return mfcc_df

In [29]:
# train_x = get_mfcc_feature(train_df)
# test_x = get_mfcc_feature(test_df)

In [30]:
# train_y = train_df['label']

In [32]:
train_df.to_csv("train_df_sst.csv")

In [33]:
test_df.to_csv("test_df_sst.csv")